In [0]:
%run /Workspaces/dars_nic_391419_j3w9t_collab/CCU013_02/atlas/00_Load_helpers

In [0]:
%sql 
select * from  dars_nic_391419_j3w9t_collab.gdppr_dars_nic_391419_j3w9t_archive g 
WHERE g.ProductionDate ==  "2022-01-20 14:58:52.353312" 
limit 5;

In [0]:
%sql 
select g.NHS_NUMBER_DEID, g.PRACTICE 
from  dars_nic_391419_j3w9t_collab.gdppr_dars_nic_391419_j3w9t_archive g 
WHERE g.ProductionDate ==  "2022-01-20 14:58:52.353312" 
and g.PRACTICE is NULL
group by NHS_NUMBER_DEID, PRACTICE;

#### Identify gp practices (1) in denominator: 


In [0]:
##subset production
ccu013_02_gdppr_prev = spark.sql(f'''
select g.NHS_NUMBER_DEID, g.DATE, g.PRACTICE
from dars_nic_391419_j3w9t_collab.gdppr_dars_nic_391419_j3w9t_archive g  
inner join dars_nic_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort c
on g.NHS_NUMBER_DEID = c.NHS_NUMBER_DEID
WHERE g.ProductionDate ==  "2022-01-20 14:58:52.353312"''')

In [0]:
ccu013_02_gdppr_prev.createOrReplaceGlobalTempView('ccu013_02_gdppr_prev') 

In [0]:
# find max event date
ccu013_02_gdppr_max_date = spark.sql(f'''
select p.NHS_NUMBER_DEID, max(p.DATE) as max_DATE
from global_temp.ccu013_02_gdppr_prev p
group by p.NHS_NUMBER_DEID''')

In [0]:
ccu013_02_gdppr_max_date.createOrReplaceGlobalTempView('ccu013_02_gdppr_max_date') 

In [0]:
ccu013_02_gdppr_max_date.count()

In [0]:
#find gp practice with max event date (most reecent) 
ccu013_02_gdppr_practice = spark.sql(f'''
select p.NHS_NUMBER_DEID, p.PRACTICE, p.DATE from global_temp.ccu013_02_gdppr_prev p
inner join
global_temp.ccu013_02_gdppr_max_date t2
on p.NHS_NUMBER_DEID = t2.NHS_NUMBER_DEID 
where p.DATE = t2.max_DATE
group by p.NHS_NUMBER_DEID, p.PRACTICE, p.DATE''')

In [0]:
ccu013_02_gdppr_practice.createOrReplaceGlobalTempView('ccu013_02_gdppr_practice') 

In [0]:
ccu013_02_gdppr_practice.count()

In [0]:
drop_table("ccu013_02_gdppr_practice")

In [0]:
create_table("ccu013_02_gdppr_practice")

In [0]:
%sql
select * from dars_nic_391419_j3w9t_collab.ccu013_02_gdppr_practice limit 4;

In [0]:
%sql 
select count(distinct(NHS_NUMBER_DEID)) from dars_nic_391419_j3w9t_collab.ccu013_02_gdppr_practice;

In [0]:
%sql 
select count(*) from dars_nic_391419_j3w9t_collab.ccu013_02_gdppr_practice;

See below for another join with ccu013_02 cohort 

### Create table with ICB (highlevel geo code), ICB name and practice code 

from Ivy: 

#### List is from:
https://cms.nhsbsa.nhs.uk/sicbls-icbs-and-other-providers/organisation-and-prescriber-changes/icbs

Statutory ICSs will comprise two key bodies 
- an integrated care board (ICB) and 
  - ICBs will take on the NHS planning functions previously held by clinical commissioning groups
- integrated care partnership (ICP). 
  - This wider strategy will be developed by the ICP. 
  - ICPs are statutory committees bringing together:
    -the NHS, 
    - local authorities and
    - other organisations, including those from social care, as equal partners to focus more widely on health, public health and social care

From 1st July 2022, Integrated Care Boards (ICBs) have replaced Strategic Information Partnerships (STPs). Clinical Commissioning Groups (CCGs) are now Sub-ICB Locations (SICBLs).

Each Integrated Care System (ICS) will have an Integrated Care Board (ICB), a statutory organisation bringing the NHS together locally to improve population health and establish shared strategic priorities within the NHS. When ICBs are legally established, clinical commissioning groups (CCGs) will be abolished

remove text of ICB code and their name to export

In [0]:
csv_text_file = """
ICB_Code,ICB_Name

"""

In [0]:
import io
import pandas as pd

df = pd.read_csv(io.StringIO(csv_text_file))

In [0]:
%py
#CONVERT DATA TYPES
df = df.convert_dtypes()
df.dtypes

In [0]:
spark.createDataFrame(df).createOrReplaceGlobalTempView('ccu013_02_icb_names')

### GP Codes and High level geo codes (ICB) from:
#### https://digital.nhs.uk/services/organisation-data-service/file-downloads/gp-and-gp-practice-related-data
#### Epraccur file

In [0]:
%py
csv_text_file="""
PRAC_CODE,PRAC_NAME,NATIONAL_GROUPING_CODE,HIGH_LEVEL_HEALTH_GEO_CODE,POSTCODE,OpenDate,CloseDate,StatusCode,AmdendedRecordIndicator,Provider_Purchaser,Prescribing_Setting

"""

In [0]:
%py
#CONVERT DATA TYPES
df = df.convert_dtypes()
df.dtypes

In [0]:
from pyspark.sql.types import *
DateType
df_schema = StructType([\
  StructField("PRAC_CODE",StringType(), True),\
                      StructField("PRAC_NAME",StringType(), True),\
                      StructField("NATIONAL_GROUPING_CODE",StringType(), True),\
                      StructField("HIGH_LEVEL_HEALTH_GEO_CODE",StringType(), True),\
                      StructField("POSTCODE",StringType(), True),\
                      StructField("OpenDate",DateType(), True),\
                      StructField("CloseDate",DateType(), True),\
                      StructField("StatusCode",StringType(), True),\
                      StructField("AmdendedRecordIndicator",StringType(), True),\
                      StructField("Provider_Purchaser",StringType(), True),\
                      StructField("Prescribing_Setting", StringType(), True)])

In [0]:
%py
spark.createDataFrame(df, schema=df_schema).createOrReplaceGlobalTempView('ccu013_02_gp_prac_high_geo_code')

#### Join gppractice and icb code (highlevelgeocode) and icb name

In [0]:
%sql 
select i.ICB_Name, i.ICB_Code from global_temp.ccu013_02_icb_names i
inner join global_temp.ccu013_02_gp_prac_high_geo_code h
where h.HIGH_LEVEL_HEALTH_GEO_CODE = i.ICB_Code
GROUP BY h.HIGH_LEVEL_HEALTH_GEO_CODE, i.ICB_Name, i.ICB_Code;

In [0]:
%sql 
select i.ICB_Name, i.ICB_Code, h.PRAC_CODE from global_temp.ccu013_02_icb_names i
inner join global_temp.ccu013_02_gp_prac_high_geo_code h
where h.HIGH_LEVEL_HEALTH_GEO_CODE = i.ICB_Code
GROUP BY h.HIGH_LEVEL_HEALTH_GEO_CODE, i.ICB_Name, i.ICB_Code, h.PRAC_CODE;

In [0]:
ccu013_02_gp_ICS_map = spark.sql(f'''
select i.ICB_Name, i.ICB_Code, h.PRAC_CODE from global_temp.ccu013_02_icb_names i
inner join global_temp.ccu013_02_gp_prac_high_geo_code h
where h.HIGH_LEVEL_HEALTH_GEO_CODE = i.ICB_Code
GROUP BY h.HIGH_LEVEL_HEALTH_GEO_CODE, i.ICB_Name, i.ICB_Code, h.PRAC_CODE
''')

In [0]:
ccu013_02_gp_ICS_map.createOrReplaceGlobalTempView('ccu013_02_gp_ICS_map') 

In [0]:
drop_table("ccu013_02_gp_ICS_map")

In [0]:
create_table("ccu013_02_gp_ICS_map")

In [0]:
%sql 
select * from dars_nic_391419_j3w9t_collab.ccu013_02_gp_ICS_map limit 5;

In [0]:
%sql 
select count(*) from dars_nic_391419_j3w9t_collab.ccu013_02_gp_ICS_map;

#### Join with our cohort:

##### Re first section of the notebooK and ccu013_02_gdppr_practice

In [0]:
%sql 
select count(distinct(PRAC_CODE)) from dars_nic_391419_j3w9t_collab.ccu013_02_gp_ICS_map;

In [0]:
%sql 
select count(distinct(PRACTICE)) from dars_nic_391419_j3w9t_collab.ccu013_02_gdppr_practice;

In [0]:
%sql 
select * from dars_nic_391419_j3w9t_collab.ccu013_02_gdppr_practice p
inner join dars_nic_391419_j3w9t_collab.ccu013_02_gp_ICS_map i
where p.PRACTICE = i.PRAC_CODE
limit 5;

In [0]:
ccu013_02_gp_ICS_map_cohort = spark.sql(f'''
select p.NHS_NUMBER_DEID, p.PRACTICE, p.DATE, i.ICB_Name, i.ICB_Code from dars_nic_391419_j3w9t_collab.ccu013_02_gdppr_practice p
inner join dars_nic_391419_j3w9t_collab.ccu013_02_gp_ICS_map i
inner join dars_nic_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort c
where p.PRACTICE = i.PRAC_CODE
and p.NHS_NUMBER_DEID = c.NHS_NUMBER_DEID
and c.SEX IN (1,2) 
AND c.IMD_quintile not like ('Unknown%')
AND c.age_23_01_2020 >= 0 and c.age_23_01_2020 <= 100
''')

In [0]:
ccu013_02_gp_ICS_map_cohort.createOrReplaceGlobalTempView('ccu013_02_gp_ICS_map_cohort') 

In [0]:
drop_table("ccu013_02_gp_ICS_map_cohort")

In [0]:
create_table("ccu013_02_gp_ICS_map_cohort")

this matches denom ccu013_02: 

In [0]:
%sql 
select count(distinct(NHS_NUMBER_DEID)) from dars_nic_391419_j3w9t_collab.ccu013_02_gp_ICS_map_cohort;

In [0]:
%sql 
select count(*) from dars_nic_391419_j3w9t_collab.ccu013_02_gp_ICS_map_cohort;

See below and script in R for tests (re multiple ids and multiple ics)

In [0]:
%sql 
select count(distinct(PRACTICE)) from dars_nic_391419_j3w9t_collab.ccu013_02_gp_ICS_map_cohort;

In [0]:
%sql 
select count(distinct(ICB_CODE)) from dars_nic_391419_j3w9t_collab.ccu013_02_gp_ICS_map_cohort;

#### Test duplicated ids ( diff practice and same ICS - ok - and with diff practice but also diff ICS - explore)

In [0]:
%sql
--N indiv with more than one practice but same ICB - not a problem for analysis as they remian within same ICS
SELECT NHS_NUMBER_DEID, ICB_Code, count(*) as n_ind_ICB
 FROM dars_nic_391419_j3w9t_collab.ccu013_02_gp_ICS_map_cohort 
 GROUP BY NHS_NUMBER_DEID, ICB_Code HAVING count(*)> 1;

In [0]:
ccu013_02_temp_n_indiv_n_ICB = spark.sql(f'''SELECT distinct(NHS_NUMBER_DEID, ICB_Code), count(*) as n_ind_ICB
 FROM dars_nic_391419_j3w9t_collab.ccu013_02_gp_ICS_map_cohort 
 GROUP BY NHS_NUMBER_DEID, ICB_Code HAVING count(*)> 1''')

In [0]:
ccu013_02_temp_n_indiv_n_ICB.count()

In [0]:
ccu013_02_temp_n_indiv_n_ICB.createOrReplaceGlobalTempView('ccu013_02_temp_n_indiv_n_ICB') 

In [0]:
drop_table("ccu013_02_temp_n_indiv_n_ICB")

In [0]:
create_table("ccu013_02_temp_n_indiv_n_ICB")

comparing  ccu013_02_temp_n_indiv_n_ICB and dars_nic_391419_j3w9t_collab.ccu013_02_gp_ICS_map_cohort 


In [0]:
%sql 
select count(distinct(NHS_NUMBER_DEID)) from dars_nic_391419_j3w9t_collab.ccu013_02_gp_ICS_map_cohort 
where ICB_Code is NULL;

For the analyses in R:

ccu013_02_gp_ICS_map_cohort

--test N practices in gp raw table production date:

In [0]:
%sql 
select count(distinct(g.PRACTICE))
from dars_nic_391419_j3w9t_collab.gdppr_dars_nic_391419_j3w9t_archive g 
inner join dars_nic_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort c 
on g.NHS_NUMBER_DEID = c.NHS_NUMBER_DEID
WHERE g.ProductionDate ==  "2022-01-20 14:58:52.353312"
and datediff(to_date('2020-01-23', 'yyyy-MM-dd'), g.DATE) > 0;

In [0]:
%sql 
select m.ICB_CODE, c.SEX, count(distinct(m.NHS_NUMBER_DEID))as n_indiv from dars_nic_391419_j3w9t_collab.ccu013_02_gp_ICS_map_cohort m
inner join dars_nic_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort c
on m.NHS_NUMBER_DEID = c.NHS_NUMBER_DEID
WHERE c.SEX IN (1,2) 
AND c.IMD_quintile not like ('Unknown%')
AND c.age_23_01_2020 >= 0 and c.age_23_01_2020 <= 100
group by m.ICB_CODE, c.SEX
order by ICB_CODE;

In [0]:
ccu013_02_gp_ICS_map_cohort_deno = spark.sql(f'''
select m.ICB_CODE, c.SEX, count(distinct(m.NHS_NUMBER_DEID))as n_indiv from dars_nic_391419_j3w9t_collab.ccu013_02_gp_ICS_map_cohort m
inner join dars_nic_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort c
on m.NHS_NUMBER_DEID = c.NHS_NUMBER_DEID
WHERE c.SEX IN (1,2) 
AND c.IMD_quintile not like ('Unknown%')
AND c.age_23_01_2020 >= 0 and c.age_23_01_2020 <= 100
group by m.ICB_CODE, c.SEX
order by ICB_CODE''')

In [0]:
ccu013_02_gp_ICS_map_cohort_deno.createOrReplaceGlobalTempView('ccu013_02_gp_ICS_map_cohort_deno') 

In [0]:
drop_table("ccu013_02_gp_ICS_map_cohort_deno")

In [0]:
create_table("ccu013_02_gp_ICS_map_cohort_deno")

In [0]:
%sql 
select c.SEX, count(distinct(m.NHS_NUMBER_DEID))as n_indiv from dars_nic_391419_j3w9t_collab.ccu013_02_gp_ICS_map_cohort m
inner join dars_nic_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort c
on m.NHS_NUMBER_DEID = c.NHS_NUMBER_DEID
WHERE c.SEX IN (1,2) 
AND c.IMD_quintile not like ('Unknown%')
AND c.age_23_01_2020 >= 0 and c.age_23_01_2020 <= 100
group by c.SEX;